In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
! pip install tsfresh


     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 675kB 30.3MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=e66847fc2e9460926f8546c6c0d1f574e576030589c47757f056cb536c96172f
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
ERROR: distributed 2021.2.0 has requirement dask>=2021.02.0, but you'll have dask 2.12.0 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


In [ ]:
df= pd.read_excel('final.xlsx')


In [ ]:
df.head()

,Unnamed: 0,Day,Day of week,Cost,Impressions,Clicks,CTR,Avg. CPC,Conversions,Conv. rate,Cost / conv.,Search Impr. share,Search Lost IS (rank),Search Lost IS (budget),Zip,Income,lat,lng
0,0,2019-01-02,2,1298.05,44533,54,0.0012,24.04,15.0,0.0540,86.54,0.3500,0.0935,0.5565,92392,104993.5,34.479332,-117.397422
1,1,2019-01-03,3,1480.26,2221,59,0.0266,25.09,10.1,0.1712,146.56,0.5233,0.1120,0.3647,90003,40865.2,33.963074,-118.274809
2,2,2019-01-04,4,1953.52,36095,110,0.0030,17.76,22.0,0.0295,85.93,0.6041,0.2028,0.1931,92201,91188.0,33.711551,-116.236141
3,3,2019-01-05,5,2507.57,18171,123,0.0068,20.39,29.5,0.0127,85.00,0.7161,0.1884,0.0955,92625,55627.0,33.601873,-117.866850
4,4,2019-01-06,6,1591.91,15185,88,0.0058,18.09,16.5,0.0087,96.48,0.7447,0.2129,0.0424,92596,12713.0,33.638018,-117.088880


In [ ]:
df= df.drop(['Unnamed: 0','lat','lng'],axis=1)

In [ ]:
df= df.drop(['Cost','Impressions'],axis=1)

In [ ]:
df['Day']=pd.to_datetime(df['Day'])
df['day']=pd.to_datetime(df['Day']).dt.day
df['month']=pd.to_datetime(df['Day']).dt.month
df['year']=pd.to_datetime(df['Day']).dt.year
df['quarter']=pd.to_datetime(df['Day']).dt.quarter

In [ ]:
df=df.drop(['Day'],axis=1)

In [ ]:
Mean_encoded_subject = df.groupby(['Zip'])['Income'].mean().to_dict() 
  
df['Zip_income'] =  df['Zip'].map(Mean_encoded_subject) 

In [ ]:
df.head()

,Day of week,Clicks,CTR,Avg. CPC,Conversions,Conv. rate,Cost / conv.,Search Impr. share,Search Lost IS (rank),Search Lost IS (budget),Zip,Income,day,month,year,quarter,Zip_income
0,2,54,0.0012,24.04,15.0,0.0540,86.54,0.3500,0.0935,0.5565,92392,104993.5,2,1,2019,1,80940.71000
1,3,59,0.0266,25.09,10.1,0.1712,146.56,0.5233,0.1120,0.3647,90003,40865.2,3,1,2019,1,40865.20000
2,4,110,0.0030,17.76,22.0,0.0295,85.93,0.6041,0.2028,0.1931,92201,91188.0,4,1,2019,1,88801.65125
3,5,123,0.0068,20.39,29.5,0.0127,85.00,0.7161,0.1884,0.0955,92625,55627.0,5,1,2019,1,55627.00000
4,6,88,0.0058,18.09,16.5,0.0087,96.48,0.7447,0.2129,0.0424,92596,12713.0,6,1,2019,1,34731.76000


In [ ]:
df=df.drop(['Zip'],axis=1)


In [ ]:
df=df.reset_index()

In [ ]:
df.head()

,index,Day of week,Clicks,CTR,Avg. CPC,Conversions,Conv. rate,Cost / conv.,Search Impr. share,Search Lost IS (rank),Search Lost IS (budget),Income,day,month,year,quarter,Zip_income
0,0,2,54,0.0012,24.04,15.0,0.0540,86.54,0.3500,0.0935,0.5565,104993.5,2,1,2019,1,80940.71000
1,1,3,59,0.0266,25.09,10.1,0.1712,146.56,0.5233,0.1120,0.3647,40865.2,3,1,2019,1,40865.20000
2,2,4,110,0.0030,17.76,22.0,0.0295,85.93,0.6041,0.2028,0.1931,91188.0,4,1,2019,1,88801.65125
3,3,5,123,0.0068,20.39,29.5,0.0127,85.00,0.7161,0.1884,0.0955,55627.0,5,1,2019,1,55627.00000
4,4,6,88,0.0058,18.09,16.5,0.0087,96.48,0.7447,0.2129,0.0424,12713.0,6,1,2019,1,34731.76000


In [ ]:
y= df['Income']
df=df.drop(['Income'],axis=1)

In [ ]:
from tsfresh.utilities.dataframe_functions import roll_time_series
df_rolled = roll_time_series(df, column_id="index")



Rolling:   0%|          | 0/1 [00:00<?, ?it/s]

Rolling: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


In [ ]:

df_rolled

,index,Day of week,Clicks,CTR,Avg. CPC,Conversions,Conv. rate,Cost / conv.,Search Impr. share,Search Lost IS (rank),Search Lost IS (budget),day,month,year,quarter,Zip_income,sort,id
0,0,2,54,0.0012,24.04,15.0,0.0540,86.54,0.3500,0.0935,0.5565,2,1,2019,1,80940.71000,0,"(0, 0)"
1,1,3,59,0.0266,25.09,10.1,0.1712,146.56,0.5233,0.1120,0.3647,3,1,2019,1,40865.20000,1,"(1, 0)"
2,2,4,110,0.0030,17.76,22.0,0.0295,85.93,0.6041,0.2028,0.1931,4,1,2019,1,88801.65125,2,"(2, 0)"
3,3,5,123,0.0068,20.39,29.5,0.0127,85.00,0.7161,0.1884,0.0955,5,1,2019,1,55627.00000,3,"(3, 0)"
4,4,6,88,0.0058,18.09,16.5,0.0087,96.48,0.7447,0.2129,0.0424,6,1,2019,1,34731.76000,4,"(4, 0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,753,6,101,0.0254,13.02,5.6,0.0554,234.77,0.3410,0.3207,0.3383,31,1,2021,1,60889.69500,753,"(753, 0)"
754,754,0,147,0.0272,17.34,25.5,0.1723,99.94,0.2888,0.2702,0.4410,1,2,2021,1,97445.96750,754,"(754, 0)"
755,755,1,131,0.0271,20.48,17.0,0.1298,157.81,0.2807,0.3802,0.3391,2,2,2021,1,154541.03600,755,"(755, 0)"
756,756,2,187,0.0308,20.22,25.9,0.1385,145.98,0.3610,0.3861,0.2528,3,2,2021,1,120925.74000,756,"(756, 0)"


In [ ]:
from tsfresh import extract_features
df_features = extract_features(df_rolled, column_id="id")





Feature Extraction:   0%|          | 0/5 [00:00<?, ?it/s]



Feature Extraction:  20%|██        | 1/5 [01:00<04:00, 60.01s/it]



Feature Extraction:  40%|████      | 2/5 [01:58<02:58, 59.51s/it]



Feature Extraction:  60%|██████    | 3/5 [02:56<01:58, 59.10s/it]



Feature Extraction:  80%|████████  | 4/5 [03:55<00:58, 58.97s/it]



Feature Extraction: 100%|██████████| 5/5 [04:53<00:00, 58.69s/it]


In [ ]:
df_features.shape


(758, 13243)

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
df_features.shape


(758, 13243)

In [ ]:
y

0      104993.50
1       40865.20
2       91188.00
3       55627.00
4       12713.00
         ...    
753      3638.60
754    105791.21
755    108946.38
756    152179.88
757     62872.40
Name: Income, Length: 758, dtype: float64

In [ ]:
features_filtered_direct.shape

(758, 242)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_features, y, test_size=0.20, random_state=0)

In [ ]:
import xgboost as xgb
xgbst_reg=xgb.XGBRegressor()

In [ ]:
xgbst_reg.fit(X_train,y_train)

[16:29:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
xgbst_reg.score(X_train,y_train)

0.8821465686455333

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(xgbst_reg,X_train,y_train,cv=5)

[16:29:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:29:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:29:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:29:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:29:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
score.mean()

0.6588089149006416

In [ ]:
pre= xgbst_reg.predict(X_test)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, pre))
print('MSE:', metrics.mean_squared_error(y_test, pre))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, pre)))

MAE: 26114.26115847939
MSE: 1176417749.5705638
RMSE: 34298.946770572475


In [ ]:
import hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials


In [ ]:

param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
xgb_model = xgb.XGBRegressor()

gsearch = GridSearchCV(estimator = xgb_model,param_grid = param_tuning,                       
                           scoring = 'neg_root_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)


In [ ]:
gsearch.fit(X_train,y_train)


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:
gsearch.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 3,
 'n_estimators': 500,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [ ]:
xgb_model_tun = xgb.XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.7,
        learning_rate = 0.01,
        max_depth = 3,
        min_child_weight = 3,
        n_estimators = 500,
        subsample = 0.5)

In [ ]:
xgb_model_tun.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=3, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.5, verbosity=1)

In [ ]:
xgb_model_tun.score(X_train,y_train)

0.8111248826836837

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(xgb_model_tun,X_train,y_train,cv=5)

In [ ]:
score.mean()

0.6693718249557319

In [ ]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


In [ ]:
!pip install catboost

     |████████████████████████████████| 65.8MB 44kB/s 


In [ ]:
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials


class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = X_train
        self.x_test  = X_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}
        

In [ ]:
obj = HPOpt(X_train, X_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=500)


Streaming output truncated to the last 5000 lines.



 23%|██▎       | 115/500 [10:59<27:56,  4.36s/it, best loss: 32437.689656321672]



 23%|██▎       | 116/500 [11:02<25:33,  3.99s/it, best loss: 32437.689656321672]



[17:02:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.




 23%|██▎       | 116/500 [11:03<25:33,  3.99s/it, best loss: 32437.689656321672]



 23%|██▎       | 117/500 [11:07<28:35,  4.48s/it, best loss: 32437.689656321672]



[17:02:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.




 23%|██▎       | 117/500 [11:08<28:35,  4.48s/it, best loss: 32437.689656321672]



 24%|██▎       | 118/500 [11:12<28:14,  4.44s/it, best loss: 32437.689656321672]



[17:02:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.




 24%|██▎       | 118/500 [11:12<28:14,  4.44s/it,

In [ ]:
xgb_opt

({'colsample_bytree': 3,
  'learning_rate': 1,
  'max_depth': 9,
  'min_child_weight': 6,
  'subsample': 0.8484051440964399},
 <hyperopt.base.Trials at 0x7f05893dd3c8>)

In [ ]:
tuned= xgb.XGBRegressor(colsample_bytree=0.3, learning_rate= 1, max_depth= 9, min_child_weight= 6, subsample=0.84)

In [ ]:
tuned.fit(X_train,y_train)

[17:53:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=1, max_delta_step=0,
             max_depth=9, min_child_weight=6, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.84, verbosity=1)

In [ ]:
tuned.score(X_train,y_train)

0.9999999999109368

In [ ]:
tuned.score(X_test,y_test)

0.3350870969722143